In [5]:
import csv
import pandas as pd
import numpy as np
import math

import folium
import folium.plugins as plugins
from folium.plugins import FloatImage
from folium.plugins import HeatMapWithTime
import ipywidgets as widgets
from IPython.display import IFrame

reload(folium)
reload(folium.plugins)

#parse and load csv file into a Pandas DataFrame
csvfile = 'Divisions.csv'
df = pd.read_csv(csvfile,parse_dates=['DATE'])

#first get all possible map dates
date_group_div = df.groupby(['DATE'])

all_dates = date_group_div.groups.keys()

#group the data by Army group and num
grouped_div = df.groupby(['Unit Type','Number'])

#create a dictionary of the army groups and nums to use in a selection widget
army_groups = dict()
for group_keys in grouped_div.groups.keys():
    army_group = group_keys[0]
    div_num = group_keys[1]
    if isinstance(div_num,str):
        if army_group in army_groups:
            army_groups[army_group].append(div_num)
        else:
            army_groups[army_group] = ['ALL',div_num]

#output widget for the map
out = widgets.Output()

#selection widgets for army group and num
group_sel = widgets.Dropdown(description='Army Group',options=['ALL']+army_groups.keys())
div_sel = widgets.Dropdown(description='Division #',options=['ALL'])

#event handler
def on_group_sel(change):
    if change['new'] in army_groups:
        div_sel.options = army_groups[change['new']]
    else:
        div_sel.options = ['ALL']
    plot_div_movement(change['new'],div_sel.value)

group_sel.observe(on_group_sel,'value')

def on_div_sel(change):
    if change['new'] is not None:
        plot_div_movement(group_sel.value,change['new'])
    
div_sel.observe(on_div_sel,'value')

known_div_types = ['MOT','INF','SEC','PZR','CAV']

def plot_div_movement(army_group,div_num):

    features = []

    heat_data = dict()

    for key, item in grouped_div:    
        
        if army_group != 'ALL':
            if div_num != 'ALL':
                if key[0] != army_group or key[1] != div_num:
                    continue
            else:
                if key[0] != army_group:
                    continue
    
        divID = '%s-%s' % (key[0],key[1])
    
        div_df = grouped_div.get_group(key)[['POINT_Y','POINT_X','DATE','Unit Type','Type_D','Weight_D','Weight_ND']]
    
        try:
            div_types = div_df['Type_D'].unique()
            div_type = div_types[0]
            if div_type not in known_div_types:
                div_type = 'UNKNOWN'
        except:
            div_type = 'UNKNOWN'
    
        icon_url = 'https://github.com/rkalyanapurdue/smolensk/raw/master/division-icons/%s.png' % div_type
    
        div_data = dict()
    
        for row in div_df.itertuples():
            data = dict()
            data['POINT_Y'] = row.POINT_Y
            data['POINT_X'] = row.POINT_X
            data['Weight_D'] = row.Weight_D
            data['Weight_ND'] = row.Weight_ND
            if row.DATE not in div_data:
                div_data[row.DATE] = [data]
            else:
                div_data[row.DATE].append(data)
            
        #backfill dates that don't exist with the previous location
        prev_locs = None
        start_date = sorted(all_dates)[0]
        end_date = sorted(all_dates)[len(all_dates)-1]
        for date in pd.date_range(start_date,end_date):
            if date not in div_data:
                if prev_locs is not None:
                    div_data[date] = prev_locs
            else:
                prev_locs = []
                for data in div_data[date]:
                    prev_locs.append(data)

        for key in sorted(div_data):
            if key not in heat_data:
                heat_data[key] = []
            for row in div_data[key]:
                size_num = 0
                if math.isnan(row['Weight_D']):
                    size_num = float(row['Weight_ND'])
                else:
                    size_num = row['Weight_D']
                size = 'Size: %.f' % round(size_num,0)
                time = '%s 23:59:59' % str(key)[:10]
                data = {
                      "type": "Feature",
                      "geometry": {
                          "type": "MultiPoint",
                          "coordinates": [[row['POINT_X'],row['POINT_Y']]]
                          },
                      "properties":{
                          "icon": "marker",
                          "divID": divID,
                          "size": size,
                          "size_num": round(size_num,0),
                          "iconstyle": {
                              "iconSize": [10, 10], 
                              "iconUrl": icon_url}, 
                          "times": [time]}
                    }
                features.append(data)

    division_map = folium.Map([54.78, 32.04], tiles='Stamen Terrain', zoom_start=7)

    plugins.TimestampedGeoJson({
        'type': 'FeatureCollection',
        'features': features
    }, transition_time=400, add_last_point=False, date_options='YYYY-MM-DD').add_to(division_map)
    
    out.clear_output()
    division_map.save('smolensk.html')
    iframe = IFrame('smolensk.html', width='100%', height='750px')
    
    with out:
        display(iframe)


plot_div_movement('ALL','ALL')

#output UI
sel_ui = widgets.HBox([group_sel,div_sel])
res = widgets.VBox([sel_ui,out])
res

VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oRHJvcGRvd24oZGVzY3JpcHRpb249dSdBcm15IEdyb3VwJywgb3B0aW9ucz0oJ0FMTCcsICdJbmYgRGl2JywgJ01vdCBSZ3QnLCAnUHpEaXbigKY=
